In [ ]:
## Google Drive에 연결
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
## pwd : /content/drive/My Drive/Colab Notebooks/samplebait

## 모델에 바로 저장해서 load
!pip install flask torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [ ]:
!pip install flask
!pip install ngrok
!pip install pyngrok
!pip install flask_ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.2 MB/s eta 0:00:00


In [ ]:
from pyngrok import ngrok
AUTH_TOKEN = "2tZphsq6YON5WdcRN6sWph8myF0_7Mk72ZnK7h8x42QkSS85P"
# ngrok 토큰 설정 (한 번만 실행하면 됨)
ngrok.set_auth_token(AUTH_TOKEN)  # 🔴 여기에 ngrok 토큰을 입력하세요


In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import torch
import logging
from transformers import BertTokenizer, BertForSequenceClassification

app = Flask(__name__)
run_with_ngrok(app)  # ngrok 실행

# 로깅 설정 추가 (Colab에서 로그를 보기 위해)
logging.basicConfig(level=logging.INFO)

# Google Drive에서 모델 및 토크나이저 불러오기
MODEL_PATH = "/content/drive/My Drive/Colab Notebooks/samplebait/clickbait_model.pt"
TOKENIZER_PATH = "/content/drive/My Drive/Colab Notebooks/samplebait/tokenizer_dir"

tokenizer = BertTokenizer.from_pretrained(TOKENIZER_PATH)

# 모델 설정
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.load_state_dict(torch.load(MODEL_PATH, map_location=torch.device('cpu')))
model.eval() ## 평가 모드

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 🔹 클릭베이트 예측 함수
def predict_clickbait(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    probabilities = torch.softmax(outputs.logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()

    label_map = {0: "Non-Clickbait", 1: "Clickbait"}
    return {
        "text": text,
        "prediction": label_map[predicted_class],
        "probabilities": probabilities.squeeze().tolist()
    }

# 🔹 API 엔드포인트 추가 (Spring Boot에서 호출)
@app.route('/predict', methods=['POST'])
def predict():
    try:
        logging.info("📡 [Flask] 요청 도착")

        ## JSON 데이터 확인
        data = request.get_json(force=True)
        logging.info("[Flask] 받은 데이터: %s", data)

        if not data or "text" not in data:
            logging.info("[Flask] text 키가 없음")
            return jsonify({"error" : "No text provieded"}), 400


        result = predict_clickbait(data["text"])
        logging.info("[Flask] 예측 결과 : ", result)

        return jsonify(result)

    except Exception as e:
        logging.info("error")
        return jsonify({"error": str(e)}), 400

# ngrok을 사용하여 5000번 포트에 대한 터널 열기
public_url = ngrok.connect(5000)
logging.info("flask 서버 실행중")
print("Public URL:", public_url)

if __name__ == '__main__':
    app.run()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-5-b9249b0547d3>:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_global

Public URL: NgrokTunnel: "https://cac4-34-106-87-147.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://cac4-34-106-87-147.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [27/Feb/2025 08:17:17] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Feb/2025 08:42:04] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [27/Feb/2025 08:42:05] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [27/Feb/2025 08:42:14] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [27/Feb/2025 08:42:25] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [27/Feb/2025 08:42:33] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Feb/2025 08:50:11] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [27/Feb/2025 08:55:36] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Feb/2025 09:01:49] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Feb/2025 09:03:28] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Feb/2025 09:03:40] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Feb/2025 09:04:09] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Feb/2025 09:0